In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score


C:\Users\Igor\AppData\Local\Temp\ipykernel_4884\3305266647.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dataset = pd.read_csv('credito4.csv', sep=';')
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [46]:
dataset.shape

(1000, 8)

In [47]:
# Exemplo do valor não expecificado
dataset.iloc[902]

SALDO_ATUAL                        NaN
RESIDENCIADESDE                      4
IDADE                               42
OUTROSPLANOSPGTO                nenhum
DATA                        30/03/2019
ESTADOCIVIL         masculino solteiro
PROPOSITO                  carro usado
CLASSE                             bom
Name: 902, dtype: object

In [48]:
y = dataset['CLASSE']
X = dataset.iloc[:,0:7]

In [49]:
# Checkando quantos valores temos nulos
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [50]:
y.isnull().sum()

0

In [51]:
# Usando o fillna para preencher os dados faltantes
mediana_saldo_atual = X['SALDO_ATUAL'].median()
mediana_saldo_atual

2323.0

In [52]:
X['SALDO_ATUAL'].fillna(mediana_saldo_atual, inplace=True)

C:\Users\Igor\AppData\Local\Temp\ipykernel_12072\3955557194.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['SALDO_ATUAL'].fillna(mediana_saldo_atual, inplace=True)


In [53]:
# Pelo warning assumo que para o futuro devemos usar X.fillna({'SALDO_ATUAL': mediana_saldo_atual}, inplace=True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [54]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo


In [55]:
X.tail(100)

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
900,2625.0,4,43,banco,28/03/2020,masculino solteiro,carro novo
901,3485.0,4,44,nenhum,29/03/2018,masculino div/sep,carro novo
902,2323.0,4,42,nenhum,30/03/2019,masculino solteiro,carro usado
903,1386.0,2,40,nenhum,30/03/2018,masculino casado/viuvo,radio/tv
904,1278.0,1,36,nenhum,01/04/2018,masculino solteiro,radio/tv
...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,29/06/2018,fem div/cas,mobilia/equipamento
996,3857.0,4,40,nenhum,30/06/2018,masculino div/sep,carro usado
997,804.0,4,38,nenhum,03/07/2018,masculino solteiro,radio/tv
998,1845.0,4,23,nenhum,04/07/2019,masculino solteiro,radio/tv


In [56]:
# Exemplo do dado alterado
X.iloc[902]

SALDO_ATUAL                     2323.0
RESIDENCIADESDE                      4
IDADE                               42
OUTROSPLANOSPGTO                nenhum
DATA                        30/03/2019
ESTADOCIVIL         masculino solteiro
PROPOSITO                  carro usado
Name: 902, dtype: object

In [57]:
# Análise da outra coluna que contem valores não preenchidos ['ESTADOCIVIL']
# PEgando o valor de moda do conjunto
estado_civil = X.groupby(['ESTADOCIVIL']).size()

In [58]:
estado_civil

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [59]:
# 'masculino solteiro' é o valor de moda
# Já usando o fillna como requerido pelo warning do pandas
X.fillna({'ESTADOCIVIL': 'masculino solteiro'}, inplace=True)

In [60]:
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

## Parte 2

Data Binding

In [61]:
agrupado = X.groupby(['PROPOSITO']).size()

In [62]:
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [63]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO' ] = 'outros'
X.loc[X['PROPOSITO'] == 'mobilia/equipamento', 'PROPOSITO' ] = 'outros'
X.loc[X['PROPOSITO'] == 'educação', 'PROPOSITO' ] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo      234
carro usado     103
negócios         97
obras            22
outros          255
qualificação      9
radio/tv        280
dtype: int64

In [75]:
# Extração de caraterísticas da data
X['DATA'] = pd.to_datetime(X['DATA'], format="%d/%m/%Y" )
X['DATA']

Timestamp('2020-01-01 00:00:00')

In [76]:
X['DATA'][1]

Timestamp('2020-01-01 00:00:00')

In [84]:
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()
X['DIA'] = X['DATA'].dt.day


In [85]:
print(X['DIA'][1])
print(X['DIASEMANA'][1])

1
Wednesday


In [88]:
# Vamos realizar o label encoder
labelencoder = LabelEncoder()
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [90]:
X['PROPOSITO'].unique()

array(['radio/tv', 'outros', 'carro novo', 'carro usado', 'negócios',
       'obras', 'qualificação'], dtype=object)

In [89]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [91]:
X['ESTADOCIVIL'] = labelencoder.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder.fit_transform(X['DIASEMANA'])

In [92]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,6,2019,1,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,6,2020,1,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,2,4
3,7882.0,4,45,nenhum,2019-01-02,3,4,2019,1,2,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,3,6


In [94]:
# Agora vamos ver como fazer o one hot encoding
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [99]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


In [97]:
# Padronizando o z score
stdscaler = StandardScaler()
m = stdscaler.fit_transform(X.iloc[:, 0:3])
m

array([[-0.03513395,  1.04698668,  1.6392759 ],
       [-0.03512697, -0.76597727, -0.74024139],
       [-0.0351326 ,  0.14050471,  0.68746898],
       ...,
       [-0.03513448,  1.04698668,  0.1058092 ],
       [-0.03513296,  1.04698668, -0.68736323],
       [-0.03512898,  1.04698668, -0.47585058]])

In [100]:
# Concatenando os dados
X = pd.concat([X, z, pd.DataFrame(m, columns=['SALDO_ATUAL_N', 'RESIDENCIADESDE_N', 'IDADE_N'])], axis=1)

In [101]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,6,2019,1,1,5,False,True,False,-0.035134,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,6,2020,1,1,6,False,True,False,-0.035127,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,2,4,False,True,False,-0.035133,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,4,2019,1,2,6,False,True,False,-0.035124,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,3,6,False,True,False,-0.035129,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,4,2018,6,29,0,False,True,False,-0.035133,1.046987,-0.264338
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,30,2,False,True,False,-0.035130,1.046987,0.211566
997,804.0,4,38,nenhum,2018-07-03,3,6,2018,7,3,5,False,True,False,-0.035134,1.046987,0.105809
998,1845.0,4,23,nenhum,2019-07-04,3,6,2019,7,4,4,False,True,False,-0.035133,1.046987,-0.687363


In [104]:
# Vamos remover algumas colunas agora
# Começando pelo saldo atual, residencia e idade, pois foram normalizadas
# Já planos pagamento foi feito o onehot encoding
# Alguma coluna do resultado do one hot do  'OUTROSPLANOSPGTO' pois precisamos retirar uma dummy variable
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'], inplace=True)

In [105]:
X

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,3,6,2019,1,1,5,True,False,-0.035134,1.046987,1.639276
1,0,6,2020,1,1,6,True,False,-0.035127,-0.765977,-0.740241
2,3,4,2020,1,2,4,True,False,-0.035133,0.140505,0.687469
3,3,4,2019,1,2,6,True,False,-0.035124,1.046987,0.475956
4,3,0,2018,1,3,6,True,False,-0.035129,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...
995,0,4,2018,6,29,0,True,False,-0.035133,1.046987,-0.264338
996,2,1,2018,6,30,2,True,False,-0.035130,1.046987,0.211566
997,3,6,2018,7,3,5,True,False,-0.035134,1.046987,0.105809
998,3,6,2019,7,4,4,True,False,-0.035133,1.046987,-0.687363


In [106]:
# Fazendo um teste com randomforest
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.3, random_state=0)
floresta = RandomForestClassifier(n_estimators=100)
floresta.fit(X_treinamento, y_treinamento)
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)

In [107]:
taxa_acerto

0.7066666666666667

In [108]:
confusao

array([[192,  22],
       [ 66,  20]], dtype=int64)

In [111]:
previsoes[0:10]

array(['bom', 'bom', 'bom', 'ruim', 'bom', 'ruim', 'bom', 'bom', 'ruim',
       'bom'], dtype=object)

In [112]:
y_teste[0:10]

993     bom
859     bom
298     bom
553     bom
672     bom
971     bom
27      bom
231     bom
306     bom
706    ruim
Name: CLASSE, dtype: object